In [2]:
# coding=utf-8
from __future__ import print_function, absolute_import, unicode_literals
from gm.api import *

import MySQLdb
import codecs
from datetime import *
import time

set_token('a0998908534d317105b2184afbe436a4104dc51b')

class DateUtil():
    @staticmethod
    def toMysqlDatetimeStr(strtime):
        date = datetime.strptime(strtime, "%Y-%m-%dT%H:%M:%S+08:00")
        return date.strftime("%Y-%m-%d %H:%M:%S")
    
class DB():
    def __init__(self):
        self.db = MySQLdb.connect("unionfight.citypet.cn","pocketpet","3MjdvUuXqxca6cbc","goldminer",charset="utf8")
        self.cursor = self.db.cursor()
    
    def addStockDailyBar(self, bar):
        sql = """INSERT INTO bar_daily_adjust_none(code, trade_date, open, close, high, low, amount, volume, 
        adj_factor, pre_close, upper_limit, lower_limit)
         VALUES ('%s', '%s', %f, %f, %f, %f, %f, %f, %f, %f, %f, %f)"""
        sql = sql % (bar.sec_id, DateUtil.toMysqlDatetimeStr(bar.strtime), bar.open, bar.close, bar.high, bar.low, 
                     bar.amount, bar.volume, bar.adj_factor, bar.pre_close, bar.upper_limit, bar.lower_limit)
        self.cursor.execute(sql)
        self.db.commit()

    def addIndexConstituent(self, code, constituent):
        sql = """INSERT INTO index_constituents(code, trade_date, constituents) VALUES ('%s', '%s', "%s")"""
        print(constituent.keys())
        sql = sql % (code, constituent['trade_date'], constituent['constituents'])
        self.cursor.execute(sql)
        self.db.commit()
    
    def fundamentalToSql(self, code, fundamental):
        sql = "('%s', '%s' , '%s'" % (code, fundamental['pub_date'], fundamental['end_date'])
        
        fieldstr = "DY,EV,EVEBITDA,EVPS,LYDY,NEGOTIABLEMV,PB,PCLFY,PCTTM,PELFY,PELFYNPAAEI,PEMRQ,PEMRQNPAAEI,PETTM,PETTMNPAAEI,PSLFY,PSMRQ,PSTTM,TCLOSE,TOTMKTCAP,TRADEDATE,TURNRATE,TOTAL_SHARE,FLOW_SHARE"
        fields = fieldstr.split(",")
        
        for field in fields:
            if field in fundamental:
                sql += (", %lf" % fundamental[field])
            else:
                sql += ", 0"

        sql = sql + ")"
        # print(sql)
        return sql
        
    def addFundamental(self, code, fundamentals):
        fieldstr = "DY,EV,EVEBITDA,EVPS,LYDY,NEGOTIABLEMV,PB,PCLFY,PCTTM,PELFY,PELFYNPAAEI,PEMRQ,PEMRQNPAAEI,PETTM,PETTMNPAAEI,PSLFY,PSMRQ,PSTTM,TCLOSE,TOTMKTCAP,TRADEDATE,TURNRATE,TOTAL_SHARE,FLOW_SHARE"
        fields = fieldstr.split(",")
        sql = """INSERT INTO trading_derivative_indicator(code, pub_date, end_date, """ + fieldstr + """) 
        VALUES """
        
        for fundamental in fundamentals:
            sql += self.fundamentalToSql(code, fundamental) + ","
        
        sql = sql[:-1] + ";"

        # print(sql)
        self.cursor.execute(sql)
        self.db.commit()

    def getStockList(self):
        sql = "select code from stocks"
        self.cursor.execute(sql)
        self.db.commit()
        results = self.cursor.fetchall()
        stocks = []
        for row in results:
            stocks.append(row[0])
        return stocks

    def getIndexList(self):
        sql = "select code from indexes"
        self.cursor.execute(sql)
        self.db.commit()
        results = self.cursor.fetchall()
        stocks = []
        for row in results:
            stocks.append(row[0])
        return stocks
        
    def getStockStartDate(self, code):
        sql = "select pub_date from stocks where code = '%s'" % code
        self.cursor.execute(sql)
        self.db.commit()
        return self.cursor.fetchone()[0]
    
    def getStockLatestDate(self, code):
        sql = "select trade_date from bar_daily_adjust_none where code = '%s' order by trade_date desc limit 1" % code
        self.cursor.execute(sql)
        result = self.cursor.fetchone()
        if result is None:
            return None
        return result[0]

    def getLastIndexDate(self, code):
        sql = "select trade_date from index_constituents where code = '%s' order by trade_date desc limit 1" % code
        self.cursor.execute(sql)
        result = self.cursor.fetchone()
        if result is None:
            return datetime(2005, 1, 1)
        return result[0] + timedelta(days=1)

    def getLastFundamentalsDate(self, code):
        sql = "select pub_date from trading_derivative_indicator where code = '%s' order by pub_date desc limit 1" % code
        self.cursor.execute(sql)
        result = self.cursor.fetchone()
        if result is None:
            return datetime(2005, 1, 1)
        return result[0] + timedelta(days=1)

        
class StockData():
    def __init__(self):
        self.db = DB()
        
    def getStockList(self):
        return self.db.getStockList()
    
    def getSymbol(self, code):
        if code[0:1] == "6":
            return "SHSE"
        return "SZSE"
    
    def queryAllStockBar(self, md, code):
        startdate = self.db.getStockLatestDate(code)
        if startdate is None:
            startdate = self.db.getStockStartDate(code)
        else:
            startdate = startdate + timedelta(days=1)
        enddate = datetime.now()+timedelta(days=1)
        print(startdate, enddate)
        bars = md.get_dailybars(self.getSymbol(code) + "." + code, startdate.strftime("%Y-%m-%d"), enddate.strftime("%Y-%m-%d"))
        return bars
    
    def saveBars(self, bars):
        for bar in bars:
            self.db.addStockDailyBar(bar)
        print("insert %d bars" % len(bars))

class FundamentalsData():
    def __init__(self):
        self.db = DB()

    def getSymbol(self, code):
        if code[0:1] == "6":
            return "SHSE"
        return "SZSE"

    def getFundamentals(self, code):
        startdate = self.db.getLastFundamentalsDate(code)
        enddate = datetime.now()+timedelta(days=1)
        print(startdate, enddate, self.getSymbol(code)+ "." +code)
        results = get_fundamentals(table='trading_derivative_indicator', symbols=self.getSymbol(code) + "." + code, 
            start_date=startdate, end_date=enddate, 
                fields='DY,EV,EVEBITDA,EVPS,LYDY,NEGOTIABLEMV,PB,PCLFY,PCTTM,PELFY,PELFYNPAAEI,PEMRQ,PEMRQNPAAEI,PETTM,PETTMNPAAEI,PSLFY,PSMRQ,PSTTM,TCLOSE,TOTMKTCAP,TRADEDATE,TURNRATE,TOTAL_SHARE,FLOW_SHARE')
        return results

    def saveFundamentals(self, code, fundamentals):
#         for fundamental in fundamentals:
        if len(fundamentals) > 0:
            self.db.addFundamental(code, fundamentals)
            print("insert %d fundamentals" % len(fundamentals))
        else:
            print("%s is up to date" % code)

providor = FundamentalsData()
stockData = StockData()
for code in stockData.getStockList():
    print("##", code, "##")

    fundamentals = providor.getFundamentals(code)
    providor.saveFundamentals(code, fundamentals)
    # break

ModuleNotFoundError: No module named 'gm'